In [ ]:
#Description:  This is a 'Smart' chat bot program

In [ ]:
pip install nltk

In [ ]:
pip install newspaper3k

In [ ]:
#Import the libraries
from newspaper import Article
import random
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Download the punkt package
nltk.download('punkt', quiet=True)


In [ ]:
#Get the article
#article = Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
article = Article('https://en.wikipedia.org/wiki/Tour_de_France')
article.download()
article.parse()
article.nlp()
corpus = article.text


In [ ]:
#Print the articles Text
print(corpus)

In [ ]:
#Segmentation
text = corpus
sentence_list = nltk.sent_tokenize(text) #A list of sentences

In [ ]:
#Print the list of sentences
print(sentence_list)

In [ ]:
#A function to return a random greeting response to a users greeting
#An Intent  -- greeting
def greeting_response(text):
  text = text.lower()

  #Bots greeting response
  bot_greetings = ['howdy','hi','hey','hello','hola', 'yo', 'que tal']
  #Users greeting
  user_greetings = ['hi','hey','hello','hola', 'greetings', 'wassup']

  #loops through each word in text
  for word in text.split():
    if word in user_greetings:
      return random.choice(bot_greetings)


In [ ]:
def index_sort(list_var):
  length = len(list_var)
  list_index = list(range(0, length))

  x = list_var
  for i in range(length):
    for j in range(length):
      if x[list_index[i]] > x[list_index[j]]:
        #Swap
        temp = list_index[i]
        list_index[i] = list_index[j]
        list_index[j] = temp

  return list_index

In [ ]:
#Create the bots reponse
def bot_response(user_input):
    #make them all lower case
    user_input = user_input.lower()

    #adds the user_input to the sentence list
    sentence_list.append(user_input)

    #create an empty bot response
    bot_response = ''

    #Create a count matrix from the sentence list
    cm = CountVectorizer().fit_transform(sentence_list)

    #get the similarity score - compares the user_input
    #(the last line in the counter maxtrix cm[-1])
    #to everything in the matrix itself
    similarity_scores = cosine_similarity(cm[-1], cm)

    #reduce the dimensionality of the similarity_scores
    #and store it in similarity_scores_list
    similarity_scores_list = similarity_scores.flatten()

    #Create a list of the indices of similarity_scores_list
    #sorted in decending order their values
    index = index_sort(similarity_scores_list)

    #Drop the first element (the user's input) from index
    index = index[1:]

    #signals that a response was generated to the user
    response_flag = 0

    #a counter of similar sentences found
    j = 0

    for i in range(len(index)):
      #if similarity score > 0 - match found
      if similarity_scores_list[index[i]]>0.0:
        #add the found sentence to the bot's response
        bot_response = bot_response+' '+sentence_list[index[i]]
        #signal that a response was generated
        response_flag =1
        #increment the found sentences counter
        j = j+1
      #only generate 2 possible responses
      if j > 2:
        break

    #if no sentences were found
    if response_flag == 0:
      #nice way of of saying nothing found
      bot_response = bot_response+ ' '+ "I apologize, I don't understand."

    #removes the user's input in preperation for the next comparsion
    sentence_list.remove(user_input)

    return bot_response

In [ ]:
#Start the chat
print('Doc Bot: I am Doctor Bot or Doc Bot for short. I will answer your queries about Chronic Kidney Disease. If you want to exit, type bye.')

#list to break infinite loop
exit_list = ['exit','see you later', 'bye', 'quit', 'break']

#infinite loop
while(True):
  user_input = input()
  if user_input.lower() in exit_list:
    print('Doc Bot: Chat with you later!')
    break
  else:
    if greeting_response(user_input) != None:
      print('Doc Bot: '+greeting_response(user_input))
    else:
      print('Doc Bot: '+bot_response(user_input))